# Integração Power BI e Jupyter

### Integrações antigas -> Python dentro do Power BI

### Nova Integração -> Power BI dentro do Jupyter

1. Instalar a biblioteca powerbiclient
2. Importar Report e models, Autenticação
3. Realizar a Autenticação
4. Group_ID e Report_id
5. Carregar o relatório como group_id e o report_id
    - Pegar essas informações do link do Power BI
    - Precisa ter conta corporativa. Passo a Passo gratuito: https://youtu.be/JCJPD0pRxD8
    - Exemplo de link (não use esse, é só um exemplo): https://app.powerbi.com/groups/61b5c35b-9d17-9e1b0ebd3b5f/reports/82b47b44-b8ed-6b6bade73e24/ReportSection421e7a42c094ea8a1e04
6. Se der erro para carregar o relatório:
    - Aperte F12
    - Vá em configurações
    - Desative o Javascript source maps e CSS source maps
    - Reinicie o navegador

In [33]:
# !pip install powerbiclient
from powerbiclient import Report, models

# para gerar uma autenticacao com o relatorio em questao de modo a poder edita-lo
from powerbiclient.authentication import DeviceCodeLoginAuthentication

In [34]:
autenticacao = DeviceCodeLoginAuthentication()

Performing device flow authentication. Please follow the instructions below.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RE8S84C9F to authenticate.

Device flow authentication successfully completed.
You are now logged in .

The result should be passed only to trusted code in your notebook.


In [35]:
# group_id nao foi identificado na URL, mas em geral ele que é usado no lugar do embed_url, na ausencia dele podemos usar o embed_url do relatorio
# group_id='me'
# report_id='f9718127-d9fe-4d86-bda4-f6fba4fd5efe'
embed_url='https://app.powerbi.com/groups/me/reports/f9718127-d9fe-4d86-bda4-f6fba4fd5efe/ReportSection421e7a42c094ea8a1e04?experience=power-bi'
report_id='f9718127-d9fe-4d86-bda4-f6fba4fd5efe'

In [36]:
# com esse comando o relatorio completo virá para essa variavel relatorio
relatorio=Report(embed_url=embed_url,report_id=report_id,auth=autenticacao)

In [37]:
relatorio # imprime o relatorio

Report()

In [31]:
from IPython.display import IFrame
display(IFrame(src="https://app.powerbi.com/reportEmbed?reportId=d71c6356-b975-4a32-a09a-f173a40539fe&autoAuth=true&ctid=56f958aa-959f-40ce-9f84-8186316b3219&config=eyJjbHVzdGVyVXJsIjoiaHR0cHM6Ly93YWJpLWJyYXppbC1zb3V0aC1yZWRpcmVjdC5hbmFseXNpcy53aW5kb3dzLm5ldC8ifQ%3D%3D", width=1140, height=541))

### Agora podemor interagir com o relatório

In [43]:
# agora podemos utilizar o relatorio armazenado na variavel relatorio conforme for interessante
# vamos pegar as paginas do relatorio:
paginas=relatorio.get_pages()

print('Número de paginas do relatório: '+str(len(paginas)))

Exception: Power BI report is not embedded

- Como assim? Vamos entender

In [40]:
for pagina in paginas:
    print('*'*10)
    print(pagina)
    if pagina['displayName']=='Relatório de Vendas':
        pagina_relatorio_vendas=pagina

# observe na resposta que temos 13 paginas, mas apenas 3 estão visiveis (visibility: 0)
# os tooltips e os relatorios sao considerados páginas

**********
{'name': 'ReportSection4b4b44a3486a18e816b2', 'displayName': 'Relatório - Parte 1', 'isActive': False, 'visibility': 1, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}
**********
{'name': 'ReportSection9c87b2d60325309ad45c', 'displayName': 'Relatório - Parte 2', 'isActive': False, 'visibility': 1, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}
**********
{'name': 'ReportSection04f530e093ad244246ab', 'displayName': 'Relatório - Parte 3', 'isActive': False, 'visibility': 1, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': None, 'wallpaper': None}
**********
{'name': 'ReportSection329cbe48246e27f78bae', 'displayName': 'Relatório - Parte 4', 'isActive': False, 'visibility': 1, 'defaultSize': {'type': 0, 'width': 1280, 'h

### Vamos puxar agora 1 página: Relatório de Vendas 

In [41]:
print(pagina_relatorio_vendas)

# o metodo visuals_on_page() que so funciona no relatorio, devolve os graficos que existem na pagina e todos os outros tipos de 'visuais', a referencia para usa-lo deve ser sempre o campo nome
lista_graficos=relatorio.visuals_on_page(pagina_relatorio_vendas['name'])
print(len(lista_graficos))

for grafico in lista_graficos:
    print('*'*10)
    print(grafico)
    if grafico['type']=='areaChart':
        grafico_area=grafico

# observe que no par chave, valor 'type':'tipo' tipo será o tipo de grafico (area, barra, mapa, etc)

{'name': 'ReportSection421e7a42c094ea8a1e04', 'displayName': 'Relatório de Vendas', 'isActive': True, 'visibility': 0, 'defaultSize': {'type': 0, 'width': 1280, 'height': 720}, 'mobileSize': None, 'defaultDisplayOption': 0, 'background': {'color': '#FFFFFF', 'transparency': 0, 'hasImage': True}, 'wallpaper': {'color': '#FFFFFF', 'transparency': 0, 'hasImage': False}}


Exception: Power BI report is not embedded

### Pegar os gráficos de área (ou gráfico com determinada condição, como Título) 

In [24]:
print(grafico_area)

{'name': 'b429872625f46d1d6919', 'title': 'Total Faturamento Mensal', 'type': 'areaChart', 'layout': {'x': 241.89723320158106, 'y': 165.0592885375494, 'z': 4000, 'width': 717.1541501976285, 'height': 273.201581027668, 'displayState': {'mode': 0}}}


### Exportar informações do Power BI para o Python 

In [27]:
# as referencias a serem utilizadas devem ser o nome, vamos colocar rows=100 pois o numero de linhas pode ser maior, se for menor nao pegara todos os dados
infos_grafico=relatorio.export_visual_data(pagina_relatorio_vendas['name'],grafico_area['name'],rows=100)

print(infos_grafico)

Exception: Power BI report is not embedded

,Total Lucro,Categoria,Total Pedidos
0,"R$ 416800,00",Celular,299
1,"R$ 283150,00",Notebook,239
2,"R$ 175050,00",Televisão,209
3,"R$ 138700,00",Câmera,234
4,"R$ 67550,00",Smart Watch,111
5,"R$ 17600,00",Tablet,24


### Outra forma de exportar 

Total Lucro,Total Pedidos,Total Faturamento,Margem de Lucro,Meta de Faturamento,% Variacao Faturamento,Total Custos,Total Devolucoes,% de Devolucoes,Total Pedidos Ano Anterior,Meta de Devolucoes,Cor
"R$ 1098850,00",1116,"R$ 2343300,00","46,89%","R$ 2071050,00","13,15%",1244450,67,"6,00%",,33,#01B8AA



,Total Lucro,Total Pedidos,Total Faturamento,Margem de Lucro,Meta de Faturamento,% Variacao Faturamento,Total Custos,Total Devolucoes,% de Devolucoes,Total Pedidos Ano Anterior,Meta de Devolucoes,Cor
0,"R$ 1098850,00",1116,"R$ 2343300,00","46,89%","R$ 2071050,00","13,15%",1244450,67,"6,00%",NaN,33,#01B8AA


### Trabalhar com Filtros 

- Identificar se tem filtro aplicado

[{'$schema': 'http://powerbi.com/product/schema#basic', 'target': {'table': 'CadastroLojas', 'column': 'Loja'}, 'filterType': 1, 'operator': 'All', 'values': [], 'requireSingleSelection': False}]


interactive(children=(Dropdown(description='Anos', options=(2016, 2017, 2018), value=2016), Output()), _dom_cl…

<function __main__.filter_report(Anos)>

### Verificar slicers (ainda não funciona)

[{'name': 'dd0030abd0cd5cdec26d', 'type': 'slicer', 'layout': {'x': 934.8616600790515, 'y': 12.806324110671937, 'z': 9000, 'width': 237.62845849802375, 'height': 66.87747035573123, 'displayState': {'mode': 0}}}, {'name': '2c713cf5d930071a5238', 'type': 'slicer', 'layout': {'x': 643.1620553359684, 'y': 12.806324110671937, 'z': 10000, 'width': 256.12648221343875, 'height': 83.95256916996048, 'displayState': {'mode': 0}}}]


Ano
2016
2017
2018

Loja
Belo Horizonte
Campinas
Curitiba
Fortaleza
Goiânia
Guarulhos
Niterói
Nova Iguaçu
Porto Alegre
Recife
Rio de Janeiro
Salvador
São Paulo

